# VIH Noisy Dynamic Identification

In [1]:
using DrWatson
@quickactivate :ExtendedKalmanFilterNeuralTraining
#@quickactivate "ExtendedKalmanFilterNeuralTraining"; include(srcdir("ExtendedKalmanFilterNeuralTraining.jl"))

using XLSX

using LinearAlgebra: Diagonal
using Distributions: MvNormal

In [2]:
using ModelingToolkit, DifferentialEquations

@named hiv = HIV()
display(hiv)

HIV_tspan = (0.0, 365.0 * 10)
HIV_prob = ODEProblem(hiv, [], HIV_tspan)
HIV_sol = solve(HIV_prob);

x = DataFrame(hcat(map(HIV_sol, 0.0:2:max(HIV_tspan...))...)', ["T", "T_inf", "M", "M_inf", "V"]);

Model hiv with 5 equations
States (5):
  T(t) [defaults to 1000.0]
  T_inf(t) [defaults to 0.0]
  M(t) [defaults to 150.0]
  M_inf(t) [defaults to 0.0]
⋮
Parameters (15):
  ρ_T [defaults to 0.01]
  C_T [defaults to 300]
  k_T [defaults to 4.57e-5]
  s_T [defaults to 10.0]
⋮

## Training Algorithm Comparison

#### Single experiment with fixed noise.

In [3]:
SST = "Tikonov_Sample_Metrics"
SSEFK = "EFK_Sample_Metrics"
N = 5
dB = 20
σ = dBs2σ.(dB)
noise_robustness_simulations(N, x, TikonovEKF!(α=5e15), gaussian_noise, [σ], SST, names(x))
noise_robustness_simulations(N, x, EKF!(), gaussian_noise, [σ], SSEFK, names(x))

In [4]:
Tikonov_Sample_Metrics = get_sims_data(joinpath("noise_robustness",SST));
EFK_Sample_Metrics = get_sims_data(joinpath("noise_robustness",SSEFK));

In [5]:
gaussian_noisy_x = mapcols(gaussian_noise(σ), x);
RHONN, params = HIV_model(gaussian_noisy_x)
X_Tikonov = train!(RHONN, gaussian_noisy_x, params; algorithm=TikonovEKF!(α=5e15));

In [6]:
export_df(X_Tikonov, "TikonovEFK", datadir("noise_robustness", "Single_Event"))
export_df(x, "data", datadir("noise_robustness", "Single_Event"))
export_df(gaussian_noisy_x, "noisy_data", datadir("noise_robustness", "Single_Event"))

#### Sampling EFK! vs TikonovEFK! Robustness of Gaussian Noise
 

In [7]:
N = 50
dBs = 0:1:20

@time for alg in [EKF!, TikonovEKF!]
    sigmas = dBs2σ.(dBs)
    noise_robustness_simulations(N, x, alg(;α=5e15), gaussian_noise, sigmas, "$alg", names(x); path=joinpath("noise_robustness","detailed_gaussian"))
end

393.482781 seconds (3.95 G allocations: 112.179 GiB, 5.70% gc time, 0.67% compilation time)


In [8]:
joined = merge_sims(datadir("noise_robustness","detailed_gaussian"), "alg")

Dict{SubString{String}, DataFrame} with 5 entries:
  "M_inf" => 84×6 DataFrame…
  "M"     => 84×6 DataFrame…
  "T"     => 84×6 DataFrame…
  "V"     => 84×6 DataFrame…
  "T_inf" => 84×6 DataFrame…

In [9]:
export_df_dict(joined, datadir("noise_robustness","detailed_gaussian"))

#### Sampling of TikonovEFK! Noise Robustness with non Gaussian Noise. 

In [168]:
N = 10
dBs = -1:1:1

@time begin
    epsilons = dBs2ϵ.(dBs)
    noise_robustness_simulations(N, x, TikonovEKF!(;α=5e15), uniform_noise, epsilons, "UniformvsGaussian\\uniform_dBs", names(x))

    sigmas = dBs2σ.(dBs)
    noise_robustness_simulations(N, x, TikonovEKF!(;α=5e15), gaussian_noise, sigmas, "UniformvsGaussian\\gaussian_dBs", names(x));
end

 23.422951 seconds (104.11 M allocations: 3.684 GiB, 21.75% gc time, 22.09% compilation time)


In [171]:
joined = merge_sims(datadir(joinpath("noise_robustness","UniformvsGaussian")), "noise_type")

Dict{SubString{String}, DataFrame} with 5 entries:
  "M_inf" => 12×6 DataFrame…
  "M"     => 12×6 DataFrame…
  "T"     => 12×6 DataFrame…
  "V"     => 12×6 DataFrame…
  "T_inf" => 12×6 DataFrame…

In [175]:
export_df_dict(joined, datadir("noise_robustness","UniformvsGaussian"))

# Recycle

##### Ensemble Comparison

In [ ]:
using LinearAlgebra: Diagonal 
using Distributions: MvNormal

N = 5
σ = 0

control = []
results = []

for i in 1:N
    gaussian_noisy_x = mapcols(col -> max.(rand(MvNormal(col, σ * Diagonal(col.*noiseQ2))), 0.0), x);
    F, params = HIV_model(x)
    X, W = train!(F, x, params; algorithm=EKF!(), save_weights=true);
    push!(control, W)
end

for i in 1:N
    gaussian_noisy_x = mapcols(col -> max.(rand(MvNormal(col, σ * Diagonal(col.*noiseQ2))), 0.0), x);
    F, params = HIV_model(x)
    ReX, W = train!(F, x, params; algorithm=TikonovEKF!(α=2e14), save_weights=true);
    push!(results, W)
end

HIV_plot([gaussian_noisy_x, first(results), first(control)], size=(1200,900), lbl = ["Noisy Data" "TikonovEFK" "EFK"])